

> l'exécution a été réalisée sur Google Collab en package pro. En raison du traitement des images en arrays et dataframe, le ram utilisé est importante et afin de réduire les délais d'execution, l'ensemble des fichiers CSV était déjà créés et stockés dans le drive.

> L'execution de ce script doit se faire une fois ces fichiers CSV créés et présents sur le drive.

NB: Le stockage de l'ensemble des fichiers contenant plus de 30k features (pixels) necessite un espace important/ Des optimisations sont evidemment necessaires pour maximiser la ressource disponible




In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)



Mounted at /content/drive


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [4]:
##Création du dataframe Train 
df_train=pd.read_csv('drive/MyDrive/Projet Son/df_train.csv',header=0,index_col=0)

In [5]:
#Encodage des machines pour créer les classes 
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df_train['class'] = LE.fit_transform(df_train['Machine'])

In [6]:
#La realisation d'un pca et/ou l'execution de modèles sur des données de si grande dimension étant impossible même avec des ressources Ram très importantes, nous le lançons sur une partie seulement du jeu de donnée en partant du principe qu'on pourra le generaliser à l'ensemble du modèle
df_train_topca = df_train.sample(frac=0.4)

In [7]:
#Decompte des éléments présents dans la selection par classe
df_train_topca['class'].value_counts()

0    1612
2    1490
3    1338
5    1318
1    1195
4    1095
Name: class, dtype: int64

In [9]:
#Preparation des données 
target = df_train_topca['class']
data = df_train_topca.drop(['Machine','ID','file','Target_ano','class'],axis=1)

In [10]:
#Réalisation d'un PCA pour ne conserver que les features (pixel) qui génèrent de la variance.
#L'idée était de partir sur l'hypothèse qu'une majorité de pixels des spectogrammes ne sont pas utiles à l'analyse.
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.9)
pca.fit(data)
print("Nombre de composantes retenues :", pca.n_components_)

Nombre de composantes retenues : 955


In [11]:
Data_pca=pca.transform(data)

In [14]:
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(Data_pca,target,test_size = .2)

In [15]:
#Test de deux modèles SVM en utilisant un noyau polynomial
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

poly = svm.SVC(kernel='poly', degree=3, C=1).fit(X_train, y_train)

In [16]:
#prédiction sur les données de tests
poly_pred = poly.predict(X_test)


In [18]:
svm_poly_accuracy = accuracy_score(y_test, poly_pred)
svm_poly_f1 = f1_score(y_test, poly_pred, average='weighted')
print('Accuracy (Poly): ', "%.2f" % (svm_poly_accuracy*100))
print('F1 (Poly): ', "%.2f" % (svm_poly_f1*100))


Accuracy (Poly):  81.12
F1 (Poly):  81.55


In [22]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, poly_pred))
print(classification_report(y_test, poly_pred))

#les résultats sont assez bons, le modèle est assez performant sauf pour la classe 2 (Fan)

[[323   1   0   1   0   0]
 [  0 250   5   0   0   0]
 [  0   0 286   4   4   1]
 [  0   0 109 181   0   3]
 [  0   0  84   4  92   3]
 [  0   0  68  12   5 174]]
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       325
           1       1.00      0.98      0.99       255
           2       0.52      0.97      0.68       295
           3       0.90      0.62      0.73       293
           4       0.91      0.50      0.65       183
           5       0.96      0.67      0.79       259

    accuracy                           0.81      1610
   macro avg       0.88      0.79      0.81      1610
weighted avg       0.88      0.81      0.82      1610



In [33]:
df_valid=pd.read_csv('drive/MyDrive/Projet Son/df_test.csv',header=0,index_col=0)

In [40]:
df_valid['class'] = LE.fit_transform(df_valid['Machine'])
target = df_valid['class']
anomaly = df_valid['Target_ano']
data = df_valid.drop(['Machine','ID','file','Target_ano','class'],axis=1)

data_valid_pca=pca.transform(data)

In [42]:
poly_valid_pred = poly.predict(data_valid_pca)

In [44]:

print(confusion_matrix(target, poly_valid_pred))
print(classification_report(target, poly_valid_pred))  
#Les conclusions sur l'efficacité de l'identification des types de machine se confirme sur ce set de test.
#Excellents résultats sur les équipements 0 et 1, bons sur les 3, 4 et 5 et mauvais sur la 2.
#L'idée étant désormais à minima sur les machines bien reconnus de voir si le modèle nous permettrait d'identfier les anomalies


[[2456    1    2    0    0    0]
 [   0 3412   95    2    0    0]
 [   1    1 1717   94   33   29]
 [   0    2  358  462   10   24]
 [   1    0  593   93  546   57]
 [   0    1  251   50   28  549]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2459
           1       1.00      0.97      0.99      3509
           2       0.57      0.92      0.70      1875
           3       0.66      0.54      0.59       856
           4       0.88      0.42      0.57      1290
           5       0.83      0.62      0.71       879

    accuracy                           0.84     10868
   macro avg       0.82      0.75      0.76     10868
weighted avg       0.87      0.84      0.84     10868



In [45]:
df_valid["classes_predites"]=poly_valid_pred

In [46]:
df_bad_pred = df_valid[df_valid["classes_predites"]!=df_valid["class"]]

In [47]:
df_bad_pred['Target_ano'].value_counts()

1    1178
0     548
Name: Target_ano, dtype: int64

In [48]:
df_valid['Target_ano'].value_counts()
#Cela nous indique que le modèle n'a identifié que 25% des enregistrements présentant des anomalies ce qui reste faible.
# En revanche le fait que + de 66% des mauvaises prédictions soient des enregistrements présentant une anomalie laisse penser que même si le modèle ne donne pas 
#satisfaction, il laisse supposer que coupler une approche de classification efficace avec une autre approche pourrait donner de bons résultats.

1    5469
0    5399
Name: Target_ano, dtype: int64

In [54]:
#Analysons les résultats cette fois en se concentrant sur les machines 0 et 1 pour lesquelles le modèle fonctionne très bien.

df_bad_pred_zoom=df_bad_pred[df_bad_pred["class"]==0]
df_bad_pred_zoom['Target_ano'].value_counts()

1    2
0    1
Name: Target_ano, dtype: int64

In [56]:
df_valid_0=df_valid[df_valid["class"]==0]
df_valid_0['Target_ano'].value_counts()
#On observe malheureusement que même si la classification fonctionne très bien, aucune information ne peut vraiment être exploité quant à la détection d'anomalie

0    1400
1    1059
Name: Target_ano, dtype: int64